# Case Study

## Background

Heart disease is the leading cause of death globally. __Heart failure__ (HF) is the primary cause of heart diseases, and that refers to a physiological state in which the heart is unable to pump sufficiently to maintain blood flow to meet the body's need. It usually occurs because the heart has become too weak or stiff. 

HF is routinely diagnosed by:
* __Electrocardiogram (ECG)__, which measure heart's rhythm and electrical activity. From the ECG the QRS duration can be computed. An increased QRS duration, indicate dyssynchronous contraction and relaxation of the left and right ventricles and it is a marker of detection of HF subjects.
* __MRI__ or __US imaging__, which provides structural and functional information of the heart. Evaluation of the structure and the function of the ventricles can provide useful information for diagnosis and characterization of disease. 
 	 	
Based on the result of these tests, doctors use NYHA class to classify patients' heart failure according to the severity of their symptoms:

__Class I__: no limitation is experienced in any activities; there are no symptoms from ordinary activities.

__Class II__: slight, mild limitation of activity; the patient is comfortable at rest or with mild exertion.

__Class III__: marked limitation of any activity; the patient is comfortable only at rest.

__Class IV__: any physical activity brings on discomfort and symptoms occur at rest.

### Current diagnostic technique

Currently, the parameters used in clinics to identify cardiac patients are:
* __LVEDV__ which is the maximum amount of blood that heart can pump
* __LVESV__ which is the minium amount of blood that heart can pump
* __LVSV__ which is the amount of blood pumped by the left ventricle of the heart in one contraction. 
* __Ejection fraction (LVEF)__, which computes the amount of blood of the left ventricle (LV) pumps out with each contraction. A normal heart’s ejection fraction may be between 50 and 70 percent.


## Objective

The objective of this workshop is twofold:

1. Improve the quality of a pre-trained segmentation neural network to segment the left ventricle (LV) of the heart
2. Train a simple deep neural network (DNN) to classify between healthy and heart failure subjects using clinical metrics, i.e. LVEDV, LVESV, LVSV and LVEF


# 1. Image segmentaion


In this exercise we will segment the left ventricle of the heart in relatively small images using neural networks. Below is the code for setting up a segmentation network and training it. The network isn't very good, so the exercise is to improve the quality of the segmentation by improving the network. 

The data being used here is derived from the Sunnybrook Dataset (https://www.cardiacatlas.org/studies/sunnybrook-cardiac-data/) of cardiac images, filtered to contain only left ventricle myocardium segmentations and reduced in XY dimensions.

Tensorflow 2.0 documentation can be found here: https://www.tensorflow.org/beta/


## Dependencies

In [ ]:
! pip install tensorflow-gpu==2.0.0-beta1

## Imports

In [ ]:
%matplotlib inline

import os
import numpy as np
import pylab as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import StratifiedShuffleSplit


## Download dataset


In [ ]:
![ -f scd_lvsegs.npz ] ||  wget https://github.com/Warvito/MRAI_workshop_tensorflow/raw/master/scd_lvsegs.npz

data = np.load('scd_lvsegs.npz')  # load all the data from the archive

images = data['images']  # images in BHW array order
segs = data['segs']  # segmentations in BHW array order

images = images.astype(np.float32) / images.max()  # normalize images
segs = segs.astype(np.float32)

print('Images shape: ' + str(images.shape))
print('Segmentation masks shape: ' + str(segs.shape))

plt.imshow(images[13] + segs[13] * 0.25, cmap='gray')  # show image 13 with segmentation

In [ ]:
images = images.reshape(images.shape[0], 64, 64, 1)
segs = segs.reshape(segs.shape[0], 64, 64, 1)

## Split data into training and test

For this exercise, we will divide the data between train and test data, choosing to retain the last 6 cases as our test set. Typically you would want to have a training dataset, a validation dataset checked periodically during training to ensure the network continues to produce good results for data is isn't trained on, and a test dataset used to validate the training after the fact. 

In [ ]:
test_index = 6  # keep the last 6 cases for testing

# Divide the images, segmentations, and categories into train/test sets
train_images, train_segs = images[:-test_index], segs[:-test_index]
test_images, test_segs = images[-test_index:], segs[-test_index:]

## Define loss function

As loss function, for this exercise we will use the Dice loss, which measure the overlap between the ground truth segmentation and the predicted segmentation.

In [ ]:
def dice_coef_loss(y_true, y_pred, smooth=1e-5):
    """ Function to calculate the dice loss.

    Dice = (2*|X & Y|) / (|X| + |Y|)
    """   
    batch_size = tf.shape(y_true)[0]
    y_pred = tf.sigmoid(y_pred)

    y_true = tf.reshape(y_true, [batch_size, -1])
    y_pred = tf.reshape(y_pred, [batch_size, -1])

    intersection = tf.multiply(y_pred, y_true)
    sums = y_pred + y_true

    score = 2.0 * (tf.reduce_sum(intersection, axis=1) + smooth) / (tf.reduce_sum(sums, axis=1) + smooth)  
    
    return 1 - tf.cast(tf.reduce_sum(score), tf.float32) / tf.cast(batch_size, tf.float32) 

## Define segmentation network

As neural network, we used a simple autoencoder, which has a downsampling path with 2 layers using maxpooling to reduce dimensions. After the bottom layer the decoding path upsamples using upscaling layers.

However, the current network architecture is doing poorly. Modify the current network to improve the segmentation accuracy. Some of the possible solutions are:

- Add more filters
- Add layers
- Add dropout layers
- Change activation function

In [ ]:
# Segmentation network
inputs = keras.Input(shape=(64, 64, 1))

x = keras.layers.Conv2D(filters=2, kernel_size=3, padding='same', activation='linear', use_bias='False')(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(activation='relu')(x)
x = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)

x = keras.layers.Conv2D(filters=4, kernel_size=3, padding='same', activation='linear', use_bias='False')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(activation='relu')(x)
x = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)

x = keras.layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='linear')(x)

x = keras.layers.Conv2D(filters=4, kernel_size=3, padding='same', activation='linear', use_bias='False')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(activation='relu')(x)
x = keras.layers.UpSampling2D((2, 2))(x)

x = keras.layers.Conv2D(filters=2, kernel_size=3, padding='same', activation='linear', use_bias='False')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(activation='relu')(x)
x = keras.layers.UpSampling2D((2, 2))(x)

outputs = keras.layers.Conv2D(filters=1, kernel_size=3, padding='same')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

## Train network

Once we have defined the network, we need to train it. An strategy is to load the whole dataset into memory at once (setting batch size bigger than training set), but typically this isn't possible due to memory restrictions. A different strategy is to create minibatches for each train step, which means that in each iteration only few image (minibatch) would be used for training.

This training will take about ~10 minutes (on colab enviroment). The verbose was set to 0 (mute) to improve the training speed. Try verbose=1 or 2 to check in real time the model performance.

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.005)
model.compile(loss=dice_coef_loss,
              optimizer=optimizer)

In [ ]:
%%time
n_epochs = 3000
history = model.fit(train_images, train_segs,
                    batch_size=512,
                    epochs=n_epochs,
                    verbose=0)

In [ ]:
sample = (train_images[10])[np.newaxis, :]
sample_groud_truth = (train_segs[10])[np.newaxis, :]
pred_sample = model.predict(sample)

In [ ]:
# Sample an image from the training data and look at the segmentation the network predicted for it
fig, ax = plt.subplots(1, 5, figsize=(20, 5))
ax[0].set_title('Loss')
ax[0].semilogy(history.history['loss'])
ax[1].set_title('Sample Image')
ax[1].imshow(sample[0, :, :, 0])
ax[2].set_title('Sample Ground Truth')
ax[2].imshow(sample_groud_truth[0, :, :, 0])
ax[3].set_title('Sample Logits')
ax[3].imshow(pred_sample[0, :, :, 0])
ax[4].set_title('Sample Predicted Segmentation')
ax[4].imshow(pred_sample[0, :, :, 0] > 0.5)

## Test
Once the network is been train, we can now apply the test data to the network. These images were never seen by the network so how well the task is performed is an indicator of how generalized and robust the network is.

In [ ]:
testloss = model.evaluate(test_images, test_segs,
                          batch_size=10)

pred = model.predict(test_images)

seg = pred > 0.5

fig, ax = plt.subplots(1, 4, figsize=(16, 5))
ax[0].set_title('Test Image')
ax[0].imshow(test_images[0, :, :, 0])
ax[1].set_title('Ground Truth')
ax[1].imshow(test_segs[0, :, :, 0])
ax[2].set_title('Logits')
ax[2].imshow(pred[0, :, :, 0])
ax[3].set_title('Predicted Segmentation')
ax[3].imshow(seg[0, :, :, 0])

print('Test loss:', testloss)

# 2. Classification using clinical metrics 

In this part of the workshop, we aim to use the segmentations produced from the previous segmentation network to compute the following clinical metrics LVEDV, LVESV, LVSV and LVEF to classify the between healthy and heart failure subjects.

## DNN network

For classification, we will use a deep neural network classifier (DNN) with two layers with sizes 20 and 10. To evaluate the performances of the classifier we will compute accuracy, precision and recall.

In [ ]:
def compute_metrics(y_true, y_pred, target_names):
    """ Compute metrics.
    Parameters
    ----------
    y_pred: labels predicted
    y_true: ground truth labels
    target_names:  names matching the labels
    """

    bacc = balanced_accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    print('Balanced accuracy  {0:.2f}'.format(bacc))

    headers = ["precision", "recall"]

    rows = zip(target_names, precision, recall)

    digits = 2
    longest_last_line_heading = 'weighted avg'
    name_width = max(len(cn) for cn in target_names)
    width = max(name_width, len(longest_last_line_heading), digits)
    head_fmt = u'{:>{width}s} ' + u' {:>9}' * len(headers)
    report = head_fmt.format(u'', *headers, width=width)
    report += u'\n\n'
    row_fmt = u'{:>{width}s} ' + u' {:>9.{digits}f}' + u' {:>9}\n'
    for row in rows:
        report += row_fmt.format(*row, width=width, digits=digits)
    report += u'\n'

    print(report)

    return bacc * 100, precision * 100, recall * 100

In [ ]:
''' Create a DNN network for classification with two layers with sizes 20 and 10.
Plain dense neural network of linear layers using dropout and ReLU activation.
Size of the hidden layers: [20,10]
numClasses: number of classes used for the classification'''

inputs = keras.Input(shape=(4))
x = ...
x = ...
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

## Download the data
To ensure that everyone work with the same data, download the following file that contains the images segmented with the previous network optimised.

In [ ]:
![ -f data.npz ] || wget https://github.com/Warvito/MRAI_workshop_tensorflow/raw/master/data.npz
data_filename = 'data.npz'

## Compute clinical metricas and save them in a matrix called metrics
- LVEDV (maximum volume of the cardiac cycle)
- LVESV (minimum volume of the cardiac cycle)
- LVSV = LVEDV - LVESV 
- LVEF = LVSV/LVEDV*100 

Then:
- Generate a matrix ```metrics``` that concatenate LVEDV, LVESV, LVSV and LVEF

In [ ]:
# Load data
data = np.load(data_filename)

# Save fields inside data
images = data['images']  # (805, 128, 128)
segs = data['segs']  # (805, 128, 128)
case_names = data['caseNames']  # 45
case_indices = data['caseIndices']  # 45
case_type_names = ['Normal', 'Heart Failure']
case_types = data['caseTypes']  # 45
case_types[case_types == 2] = 0
case_voxel_size = data['caseVoxelSize']
is_ed_img = data['isEDImg']  # 805
seg_types = data['segTypes']  # ['Background', 'LV Pool']

# Compute LVEDV, LVESV, LVSV and LVEF
metrics = np.zeros((len(case_names), 4))
for ii, ind in enumerate(case_indices):
    ind_ed_pat = is_ed_img[ind[0]:ind[1]]
    img_pat = images[ind[0]:ind[1], :, :]
    seg_pat = segs[ind[0]:ind[1], :, :]
    img_ed = img_pat[ind_ed_pat, :, :]
    img_es = img_pat[~ind_ed_pat, :, :]
    seg_ed = seg_pat[ind_ed_pat, :, :]
    seg_es = seg_pat[~ind_ed_pat, :, :]
    dx, dy, dz = case_voxel_size[ii]
    volume_per_voxel = dx * dy * dz * 1e-3
    density = 1.05

    metrics[ii, 0] = np.sum(np.sum(seg_ed == 1, axis=1)) * volume_per_voxel
    metrics[ii, 1] = np.sum(np.sum(seg_es == 1, axis=1)) * volume_per_voxel
    metrics[ii, 2] = metrics[ii, 0] - metrics[ii, 1]
    metrics[ii, 3] = metrics[ii, 2] / metrics[ii, 0] * 100

## Visualize the data

Plot LVEDV, LVESV, LVSV and LVEF per groups, i.e. LVEDV and LVESV, LVSV and LVEF

In [ ]:
def plot_two_class_data(x, y, title, _case_type_names):
    plt.scatter(x[y == 0, 0], x[y == 0, 1], c='r', label=_case_type_names[0])
    plt.scatter(x[y == 1, 0], x[y == 1, 1], c='b', label=_case_type_names[1])
    plt.legend()
    plt.title(title)

plt.figure(figsize=[14, 5])
plt.subplot(121)
plot_two_class_data(metrics[:, 0:2], case_types, 'Healthy vs Heart Failure', case_type_names)
plt.xlabel('LVEDV')
plt.ylabel('LVESV')
plt.subplot(122)
plot_two_class_data(metrics[:, 2:], case_types, 'Healthy vs Heart Failure', case_type_names)
plt.xlabel('LVSV')
plt.ylabel('LVEF')

## Split data into training and test

Data is unbalanced (there is no the same number of patients per group). We need to ensure that in the split training/test there is balanced number of classed. To this end use StratifiedShuffleSplit from scikit-learn that will automatically ensure that (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html).

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
for train_index, test_index in sss.split(metrics, case_types):
    x_train, x_test = metrics[train_index, :], metrics[test_index, :]
    y_train, y_test = case_types[train_index], case_types[test_index]

## Train a DNN network to perform classification


In [ ]:
%%time
################
# TO COMPLETE
################
# Select the Optimizer with lr = 0.0001
optimizer = ...

# Select the loss fuction for classification
model.compile(loss=...,
              optimizer=optimizer)

n_epochs = ...
history = model.fit(x_train, y_train,
                    batch_size=...,
                    epochs=n_epochs,
                    verbose=0)

# Deploy network and compute metrics

First deploy the trained netowrk and then compute the following metrics: balanced accuracy, precision, recall


In [ ]:
################
# TO COMPLETE
################
# Compute the predicted label
y_pred = ...
# Compute metrics
compute_metrics(y_test, y_pred, ['Healthy','Heart Failure'])